In [ ]:
#. !데이터 구조 파악! 4177개의 sample, 8개의 trait, 1개의 target(1~29)


#1. 가중치 만들기 W = np.random.normal(mean, std,[init, end])
#2. 미니배치 구현 4177 -> 128개?
#3. 수치미분, 경사하강법(CEE)  --> SGD(확률적 경사 하강법)
#4. 에폭
#5. X_train, Y_train, (X_test, Y_test 얘네도 필요한가?)
#6. 모두의 딥러닝 참고

# data는 google.colab 라이브러리 이용

In [25]:
from google.colab import files

uploaded = files.upload() #test
for fn in uploaded.keys():
  print("User uploaded file \"{name}\" with length {length} bytes"\
        .format(name = fn, length=len(uploaded[fn])))

Saving data.csv to data.csv
User uploaded file "data.csv" with length 196050 bytes


In [26]:
import numpy as np
import pandas as pd
import time
import random

In [27]:
RND_MEAN = 0
RND_STD = 0.0030

LEARNING_RATE = 0.001

In [30]:
def load_abalone_dataset():
    global input_cnt, output_cnt, data
    data_abalone = pd.read_csv("data.csv", header =None)
    input_cnt, output_cnt = 10, 1

    data_sex = data_abalone.iloc[ : ,0]
    data_sex = data_sex.map({'M':np.array([1,0,0]),'F':np.array([0,1,0]) ,'I':np.array([0,0,1])})
    data_temp = data_abalone.iloc[ : ,1:9]
    data_temp[0] = data_sex
    data_temp = data_temp[[0,1,2,3,4,5,6,7,8]]
    data = data_temp

In [1]:
#가중치 만들기
class LayerNet:
  def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.0030):
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)

  def predict(self,x):
    W1, W2 = self.params['W1'], self.params['W2']
    b1, b2 = self.params['b1'], self.params['b2']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1) #sigmoid function
    a2 = np.dot(z1,W2) + b2
    
    return softmax(a2)
  
  def loss_func(self, x, t): #손실함수 - 예상값 & 정답
    y = self.predict(x)
    return cross_entroy_error(y,t)
  
  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis = 1)
    t = np.argmax(t, axis = 1)
    
    accuracy = np.sum(y==t) / float(x.shape[0]) #여기 좀만 수정
    return accuracy

  def numerical_gradient(self, x, t):
    loss_W =lambda W : self.loss_func(x,t) #practial of W
    
    #경사 하강
    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
    return grads

  def gradient(self, x, t):
    W1, W2 = self.params['W1'], self.params['W2']
    b1, b2 = self.params['b1'], self.params['b2']
    grads = {}
    
    batch_num = x.shape[0]

    #forward
    a1 = np.dot(x,W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    y = softmax(a2)

    #backword
    dy = (y - t) / batch_num
    grads['W2'] = np.dot(z1.T,dy)
    grads['b2'] = np.sum(dy, aixs = 0 )
    
    da1 = np.dot(dy, W2.T)
    dz1 = sigmoid_grad(a1) * da1
    grads['W1'] = np.dot(x.T, dz1)
    grads['b1'] = np.sum(dz1, aixs = 0)

    return grads

In [15]:
net = LayerNet(input_size = 8, hidden_size = 100, output_size = 29)

In [17]:
print(net.params['W1'].shape)  #(input_size, hidden_size)
print(net.params['b1'].shape)  #(hidden_size)
print(net.params['W2'].shape)  #(hidden_size, output_size)
print(net.params['b2'].shape)  #(output_size)

(8, 100)
(100,)
(100, 29)
(29,)


In [39]:
def training(iters_num = 10000, batch_size = 128):
  net = LayerNet(input_size = 8, hidden_size = 100, output_size = 29)
  
  train_size = x_train.shape[0] # 8
  train_loss_list = []
  train_acc_list = []
  test_acc_list = []

  # 에폭당 반복수
  iter_per_epoch = max(train_size / batch_size, 1)

  for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    #기울기 계산
    grad = net.gradient(x_batch, t_batch)

    #매개 변수 갱신
    for key in ('W1','b1','W2','b2'):
      net.params[key] -= LEARNING_RATE * grad[key]

    #학습 경과 기록
    loss = net.loss_func(x_batch, t_batch)
    train_loss_list.append(loss)

    # 1에폭당 정확도 계산
    if i % iter_per_epoch == 0:
      train_acc = network.accuracy(x_train, t_train)
      test_acc = network.accuracy(x_test, t_test)
      train_acc_list.append(train_acc)
      test_acc_list.append(test_acc)
      print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

In [31]:
load_abalone_dataset()

In [32]:
data.head()

,0,1,2,3,4,5,6,7,8
0,"[1, 0, 0]",0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,"[1, 0, 0]",0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,"[0, 1, 0]",0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,"[1, 0, 0]",0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,"[0, 0, 1]",0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [38]:
#데이터 Train:75%, Test:25%
load_abalone_dataset()
global x_train, t_train, x_test, t_test
train_data, test_data = data.iloc[ :3133, : ], data.iloc[3133: , : ]
x_train, t_train = train_data.iloc[ : , :8], train_data.iloc[ : ,8]
x_test, t_test = test_data.iloc[ : , :8], test_data.iloc[ : ,8]

In [ ]:
training() #error -> modify later